In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import html5lib

In [2]:
import numpy as np
seasons = np.arange(9,21)

urls = []
for season in seasons:
    season_url = 'https://en.wikipedia.org/wiki/The_Bachelor_(season_{})'.format(season)
    urls.append(season_url)

In [3]:
def makeSoup(url):
    response=requests.get(url)
    soup=BeautifulSoup(response.content,"lxml")
    return soup

In [4]:
def dataCollector(soup):
    import re
    # make the soup
    this = makeSoup(soup)
    
    # find the right table
    tables = this.findChildren('table')
    table = tables[1]
    
    # turn the data into a workable format
    data   = [[td.text for td in row.select('td')]
             for row in table.findAll('tr')]
    
    # create the header row and the body
    header = ['name','age', 'hometown', 'occupation', 'elimination']
    body = data[1:]
    cols   =  zip(*body)
    
    # create a dict with the data
    tbl_d  = {name:col for name, col in zip(header,cols)}
    
    # extract the season number from the original soup
    number = re.findall(r'\d+', soup)
    
    # remove brackets
    num= ''.join(number)
    
    # create a new key of seasons
    tbl_d['season'] = [num] * len(tbl_d['age'])
    
    new_names = []
    for name in tbl_d['name']:
        cleaned_name = re.sub(r'\[\d+\]', ' ', name)
        new_names.append(cleaned_name)

    tbl_d['name'] = new_names
    
    # return dictionary
    return tbl_d

In [5]:
dic = dataCollector('https://en.wikipedia.org/wiki/The_Bachelor_(season_19)')

In [6]:
frames = []

for url in urls:
    dictionary = dataCollector(url)
    frame = pd.DataFrame(dictionary)
    frames.append(frame)

combined = pd.concat(frames)

In [7]:
combined

,age,elimination,hometown,name,occupation,season
0,24,Winner,"Pembroke Pines, Florida",Jennifer Wilson,Teacher,9
1,23,Week 7,"Carlsbad, California",Sadie Murray,Publicist,9
2,25,Week 6,"Portland, Oregon",Lisa Blank,Marketing manager,9
3,24,Week 5,"Venice, Italy",Agnese Polliza,Student,9
4,23,Week 4,"Bloomingdale, Illinois",Jeanette Pawula,Teacher,9
5,22,Week 4,"Salt Lake City, Utah",Desiree Valentin,Realtor,9
6,27,Week 3,"Galveston, Texas",Jami Matzke,Event planner,9
7,28,Week 3,"Chicago, Illinois",Gina Clark,Ultrasound technician,9
8,23,Week 3,"Houston, Texas",Erica Rose,Socialite,9
9,30,Week 2,"Nelson, British Columbia",Sarah Schnare,Journalist,9
